In [5]:
# import required libriaries
from pathlib import Path
from helpers import *

In [6]:
# path to data directory
path = Path('/home/jupyter/fpl-prediction/data')

In [9]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [10]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
75575,Yoshinori_Muto,14,4,0,Newcastle United,Manchester City,0.586862,2.745756,True,0,1920,1.0
75576,Youri_Tielemans,14,3,90,Leicester City,Everton,0.880036,1.085432,True,2,1920,1.0
75577,Yves_Bissouma,14,3,68,Brighton and Hove Albion,Liverpool,0.471069,2.295280,False,2,1920,1.0
75578,Çaglar_Söyüncü,14,2,90,Leicester City,Everton,0.880036,1.085432,True,2,1920,1.0
75579,Ørjan_Nyland,14,1,0,Aston Villa,Manchester United,0.409718,1.615813,False,0,1920,1.0


In [8]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

NameError: name 'df_train' is not defined

In [12]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [13]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season
0,Aaron_Connolly,15,4,75,Brighton and Hove Albion,Arsenal,0.471069,1.481743,False,2,1920
1,Aaron_Cresswell,15,2,90,West Ham United,Wolverhampton Wanderers,0.748111,0.715505,False,0,1920
2,Aaron_Lennon,15,3,90,Burnley,Manchester City,0.426343,2.745756,True,2,1920
3,Aaron_Mooy,15,3,90,Brighton and Hove Albion,Arsenal,0.471069,1.481743,False,8,1920
4,Aaron_Ramsdale,15,1,90,Bournemouth,Crystal Palace,0.717007,0.507429,False,2,1920
5,Aaron_Wan-Bissaka,15,2,90,Manchester United,Tottenham Hotspur,1.615813,2.112945,True,2,1920
6,Abdoulaye_Doucouré,15,3,90,Watford,Leicester City,0.504425,0.880036,False,1,1920
7,Adama_Traoré,15,3,88,Wolverhampton Wanderers,West Ham United,0.715505,0.748111,True,3,1920
8,Adam_Idah,15,4,0,Norwich,Southampton,0.260847,0.508394,False,0,1920
9,Adam_Lallana,15,3,71,Liverpool,Everton,2.295280,1.085432,True,2,1920


In [14]:
df_latest.shape

(565, 11)

In [15]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [16]:
last_play_proba.head()

,player,play_proba
0,Shkodran_Mustafi,1.00
1,Héctor_Bellerín,0.75
2,Sead_Kolasinac,1.00
3,Ainsley_Maitland-Niles,1.00
4,Sokratis_Papastathopoulos,1.00


In [17]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [18]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
0,Aaron_Connolly,15,4,75,Brighton and Hove Albion,Arsenal,0.471069,1.481743,False,2,1920,1.00
1,Aaron_Cresswell,15,2,90,West Ham United,Wolverhampton Wanderers,0.748111,0.715505,False,0,1920,1.00
2,Aaron_Lennon,15,3,90,Burnley,Manchester City,0.426343,2.745756,True,2,1920,1.00
3,Aaron_Mooy,15,3,90,Brighton and Hove Albion,Arsenal,0.471069,1.481743,False,8,1920,1.00
4,Aaron_Ramsdale,15,1,90,Bournemouth,Crystal Palace,0.717007,0.507429,False,2,1920,1.00
5,Aaron_Wan-Bissaka,15,2,90,Manchester United,Tottenham Hotspur,1.615813,2.112945,True,2,1920,1.00
6,Abdoulaye_Doucouré,15,3,90,Watford,Leicester City,0.504425,0.880036,False,1,1920,1.00
7,Adama_Traoré,15,3,88,Wolverhampton Wanderers,West Ham United,0.715505,0.748111,True,3,1920,1.00
8,Adam_Idah,15,4,0,Norwich,Southampton,0.260847,0.508394,False,0,1920,1.00
9,Adam_Lallana,15,3,71,Liverpool,Everton,2.295280,1.085432,True,2,1920,1.00


In [19]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [20]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
76140,Yoshinori_Muto,15,4,0,Newcastle United,Sheffield United,0.586862,0.196171,False,0,1920,1.0
76141,Youri_Tielemans,15,3,68,Leicester City,Watford,0.880036,0.504425,True,3,1920,1.0
76142,Yves_Bissouma,15,3,0,Brighton and Hove Albion,Arsenal,0.471069,1.481743,False,0,1920,1.0
76143,Çaglar_Söyüncü,15,2,90,Leicester City,Watford,0.880036,0.504425,True,5,1920,1.0
76144,Ørjan_Nyland,15,1,0,Aston Villa,Chelsea,0.409718,1.731113,False,0,1920,1.0


In [21]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [9]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [12]:
fixtures.head(10)

,home_team,away_team,gw
150,Aston Villa,Leicester City,16
151,Bournemouth,Liverpool,16
152,Brighton and Hove Albion,Wolverhampton Wanderers,16
153,Everton,Chelsea,16
154,Manchester City,Manchester United,16
155,Newcastle United,Southampton,16
156,Norwich,Sheffield United,16
157,Tottenham Hotspur,Burnley,16
158,Watford,Crystal Palace,16
159,West Ham United,Arsenal,16


In [13]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [14]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,16,Aston Villa,Leicester City,7,13
1,16,Bournemouth,Liverpool,91,14
2,16,Brighton and Hove Albion,Wolverhampton Wanderers,36,39
3,16,Everton,Chelsea,11,8
4,16,Manchester City,Manchester United,43,1


In [18]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [19]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [20]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,16,Leicester City,Aston Villa,Ricardo Domingos_Barbosa Pereira,2,65,None,False
1,16,Leicester City,Aston Villa,Benjamin_Chilwell,2,59,75,False
2,16,Leicester City,Aston Villa,Jonny_Evans,2,51,100,False
3,16,Leicester City,Aston Villa,Christian_Fuchs,2,43,None,False
4,16,Leicester City,Aston Villa,Çaglar_Söyüncü,2,51,None,False


In [21]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,16,Aston Villa,Leicester City,Ahmed_El Mohamady,2,42,None,True
1,16,Aston Villa,Leicester City,James_Chester,2,43,100,True
2,16,Aston Villa,Leicester City,Neil_Taylor,2,44,None,True
3,16,Aston Villa,Leicester City,Kortney_Hause,2,44,100,True
4,16,Aston Villa,Leicester City,Jonathan_Kodjia,4,54,100,True


In [24]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 51.25,
 'Aaron_Cresswell': 90.0,
 'Aaron_Lennon': 20.6,
 'Aaron_Mooy': 54.0,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 88.0,
 'Abdoulaye_Doucouré': 90.0,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 35.0,
 'Adam_Masina': 63.4,
 'Adam_Smith': 64.25,
 'Adam_Webster': 71.66666666666667,
 'Adama_Traoré': 89.5,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 72.0,
 'Adrián_San Miguel del Castillo': 20.4,
 'Ahmed_El Mohamady': 22.2,
 'Ainsley_Maitland-Niles': 0.0,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 16.2,
 'Alex_Iwobi': 61.4,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 35.2,
 'Alexander_Tettey': 65.75,
 'Alexandre_Lacazette': 81.0,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 0.0,
 'Alisson_Ramses Becker': 86.25,
 'Allan_Saint-Maximin': 83.5,
 'Andre_Gray': 46.0,
 'Andreas_Christensen': 30.0,
 'Andreas_Pereira': 61.6,
 'Andrew_Robertson': 90.0,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 43.4,
 'Andros_Townsend': 50.75,
 'André Filipe_Tavares Gomes

In [27]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [28]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,16,Aston Villa,Leicester City,Ahmed_El Mohamady,2,4.2,1.00,True,0.409718,0.880036,1920,22.200000
1,16,Aston Villa,Leicester City,James_Chester,2,4.3,1.00,True,0.409718,0.880036,1920,0.000000
2,16,Aston Villa,Leicester City,Neil_Taylor,2,4.4,1.00,True,0.409718,0.880036,1920,9.000000
3,16,Aston Villa,Leicester City,Kortney_Hause,2,4.4,1.00,True,0.409718,0.880036,1920,0.000000
4,16,Aston Villa,Leicester City,Jonathan_Kodjia,4,5.4,1.00,True,0.409718,0.880036,1920,0.800000
5,16,Aston Villa,Leicester City,Ørjan_Nyland,1,4.3,1.00,True,0.409718,0.880036,1920,16.400000
6,16,Aston Villa,Leicester City,Conor_Hourihane,3,5.7,1.00,True,0.409718,0.880036,1920,48.800000
7,16,Aston Villa,Leicester City,Jack_Grealish,3,6.0,1.00,True,0.409718,0.880036,1920,90.000000
8,16,Aston Villa,Leicester City,Anwar_El Ghazi,3,5.5,0.00,True,0.409718,0.880036,1920,0.000000
9,16,Aston Villa,Leicester City,John_McGinn,3,5.8,1.00,True,0.409718,0.880036,1920,90.000000


In [29]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')